In [ ]:
import requests

# Replace with your Alpha Vantage API key
API_KEY = 'OQJ4ER1FJIKJQ2MX'

# Function to fetch stock data from Alpha Vantage
def fetch_stock_data(symbol, output_size='compact'):
    # API URL for Alpha Vantage's TIME_SERIES_DAILY endpoint
    url = f"https://www.alphavantage.co/query"
    params = {
        "function": "TIME_SERIES_DAILY",
        "symbol": symbol,
        "outputsize": output_size,  # 'compact' for last 100 data points, 'full' for full data
        "apikey": API_KEY,
        "datatype": "csv"
    }

    response = requests.get(url, params=params)

    # Check if the response was successful
    if response.status_code == 200:
        # Save the response content as a CSV file
        with open(f'{symbol}_stock_data.csv', 'wb') as file:
            file.write(response.content)
        print(f"Data saved to {symbol}_stock_data.csv")
    else:
        print(f"Error fetching data: {response.status_code}")

# Fetch data for a specific stock (e.g., AAPL for Apple)
fetch_stock_data('AAPL')


In [ ]:
import pandas as pd

# Load the stock data from the CSV file
data = pd.read_csv('AAPL_stock_data.csv')  # Ensure the filename matches your CSV

# Convert 'timestamp' column to datetime and set it as the index
data['timestamp'] = pd.to_datetime(data['timestamp'])
data.set_index('timestamp', inplace=True)

# Drop any rows with missing values
data.dropna(inplace=True)

# Save the cleaned data to a new CSV file
data.to_csv('cleaned_AAPL_stock_data.csv')

print("Cleaned data saved to cleaned_AAPL_stock_data.csv")


In [ ]:
import os
import sys

# Set the path to your Django project
sys.path.append('C:/Users/Win/Desktop/Project/Stock Project')

# Set the Django settings module
os.environ.setdefault('DJANGO_SETTINGS_MODULE', 'stock_project.settings')

# Set up Django
import django
django.setup()


In [ ]:
import os
import sys

# Set the path to your Django project
sys.path.append('C:/Users/Win/Desktop/Project/Stock Project')

# Set the Django settings module
os.environ.setdefault('DJANGO_SETTINGS_MODULE', 'stock_project.settings')

# Set up Django
import django
django.setup()


In [ ]:
import os
import sys

# Set the path to your Django project
sys.path.append('C:/Users/Win/Desktop/Project/Stock Project')

# Set the Django settings module
os.environ.setdefault('DJANGO_SETTINGS_MODULE', 'stock_project.settings')

# Set up Django
import django
django.setup()


In [ ]:
import os
import sys

# Set the path to the folder containing 'manage.py'
sys.path.append('C:/Users/Win/Desktop/Project/Stock Project')

# Set the Django settings module to 'stocks_simplified.settings'
os.environ.setdefault('DJANGO_SETTINGS_MODULE', 'stocks_simplified.settings')

# Set up Django
import django
django.setup()


In [ ]:
import os
import sys

# Set the path to the folder containing 'manage.py'
sys.path.append('C:/Users/Win/Desktop/Project/Stock Project')

# Set the Django settings module to 'stocks_simplified.settings'
os.environ.setdefault('DJANGO_SETTINGS_MODULE', 'stocks_simplified.settings')

# Set up Django
import django
django.setup()


In [ ]:
from stockdata.models import HistoricalData
import pandas as pd

# Fetch all data from HistoricalData model
data = HistoricalData.objects.all().values('stock__symbol', 'date', 'open_price', 'close_price', 'high_price', 'low_price', 'volume')

# Convert the queryset to a pandas DataFrame
df = pd.DataFrame(list(data))

# Display the data
print(df.head())


In [ ]:
from stockdata.models import HistoricalData
import pandas as pd

# Fetch all data from HistoricalData model
data = HistoricalData.objects.all().values('stock__symbol', 'date', 'open_price', 'close_price', 'high_price', 'low_price', 'volume')

# Convert the queryset to a pandas DataFrame
df = pd.DataFrame(list(data))

# Display the data
print(df.head())



In [ ]:
from stockdata.models import HistoricalData
import pandas as pd
from asgiref.sync import sync_to_async

async def fetch_data():
    # Fetch data asynchronously
    data = await sync_to_async(list)(HistoricalData.objects.values(
        'stock__symbol', 'date', 'open_price', 'close_price', 'high_price', 'low_price', 'volume'
    ))
    
    # Convert to pandas DataFrame
    df = pd.DataFrame(data)
    
    # Return or print the first few rows of the DataFrame
    return df.head()

# Call the asynchronous function
df_head = await fetch_data()
print(df_head)


In [ ]:
from stockdata.models import HistoricalData
import pandas as pd
from asgiref.sync import sync_to_async
from sklearn.preprocessing import MinMaxScaler
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense
import numpy as np

# Step 1: Fetch data from the database asynchronously
async def fetch_data():
    # Fetch the stock data
    data = await sync_to_async(list)(HistoricalData.objects.values(
        'stock__symbol', 'date', 'open_price', 'close_price', 'high_price', 'low_price', 'volume'
    ))

    # Convert to pandas DataFrame
    df = pd.DataFrame(data)
    return df

# Step 2: Prepare data for each stock symbol
def prepare_lstm_data(df, time_step=60):
    stock_symbols = df['stock__symbol'].unique()
    predictions = {}

    for symbol in stock_symbols:
        stock_data = df[df['stock__symbol'] == symbol]
        close_prices = stock_data['close_price'].values.reshape(-1, 1)

        # Normalize data
        scaler = MinMaxScaler(feature_range=(0, 1))
        scaled_data = scaler.fit_transform(close_prices)

        # Prepare training data
        x_train, y_train = [], []
        for i in range(len(scaled_data) - time_step):
            x_train.append(scaled_data[i:i+time_step, 0])
            y_train.append(scaled_data[i + time_step, 0])
        
        x_train, y_train = np.array(x_train), np.array(y_train)
        x_train = x_train.reshape(x_train.shape[0], x_train.shape[1], 1)

        # Step 3: Build and train LSTM model for each stock
        model = Sequential()
        model.add(LSTM(50, return_sequences=True, input_shape=(x_train.shape[1], 1)))
        model.add(LSTM(50, return_sequences=False))
        model.add(Dense(25))
        model.add(Dense(1))

        model.compile(optimizer='adam', loss='mean_squared_error')
        model.fit(x_train, y_train, batch_size=1, epochs=1)

        # Step 4: Predict the next day's price for each stock
        last_60_days = scaled_data[-time_step:]
        last_60_days = last_60_days.reshape(1, -1, 1)
        predicted_price = model.predict(last_60_days)
        predicted_price_actual = scaler.inverse_transform(predicted_price)

        # Store predictions
        predictions[symbol] = predicted_price_actual[0][0]

    return predictions

# Step 5: Run the prediction process
df = await fetch_data()  # Fetch the data
predictions = prepare_lstm_data(df)  # Prepare and predict

# Print predictions for all stocks
print(predictions)


In [ ]:
import pickle

with open('stock_prediction_model.pkl','wb') as model_file:
    pickle.dump(model, model_file)

In [ ]:
from stockdata.models import HistoricalData
import pandas as pd
from asgiref.sync import sync_to_async
from sklearn.preprocessing import MinMaxScaler
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense
import numpy as np

# Step 1: Fetch data from the database asynchronously
async def fetch_data():
    # Fetch the stock data
    data = await sync_to_async(list)(HistoricalData.objects.values(
        'stock__symbol', 'date', 'open_price', 'close_price', 'high_price', 'low_price', 'volume'
    ))

    # Convert to pandas DataFrame
    df = pd.DataFrame(data)
    return df

# Step 2: Prepare data for each stock symbol
def prepare_lstm_data(df, time_step=60):
    stock_symbols = df['stock__symbol'].unique()
    predictions = {}

    for symbol in stock_symbols:
        stock_data = df[df['stock__symbol'] == symbol]
        close_prices = stock_data['close_price'].values.reshape(-1, 1)

        # Normalize data
        scaler = MinMaxScaler(feature_range=(0, 1))
        scaled_data = scaler.fit_transform(close_prices)

        # Prepare training data
        x_train, y_train = [], []
        for i in range(len(scaled_data) - time_step):
            x_train.append(scaled_data[i:i+time_step, 0])
            y_train.append(scaled_data[i + time_step, 0])
        
        x_train, y_train = np.array(x_train), np.array(y_train)
        x_train = x_train.reshape(x_train.shape[0], x_train.shape[1], 1)

        # Step 3: Build and train LSTM model for each stock
        model = Sequential()
        model.add(LSTM(50, return_sequences=True, input_shape=(x_train.shape[1], 1)))
        model.add(LSTM(50, return_sequences=False))
        model.add(Dense(25))
        model.add(Dense(1))

        model.compile(optimizer='adam', loss='mean_squared_error')
        model.fit(x_train, y_train, batch_size=1, epochs=1)

        # Step 4: Predict the next day's price for each stock
        last_60_days = scaled_data[-time_step:]
        last_60_days = last_60_days.reshape(1, -1, 1)
        predicted_price = model.predict(last_60_days)
        predicted_price_actual = scaler.inverse_transform(predicted_price)

        # Store predictions
        predictions[symbol] = predicted_price_actual[0][0]

        # Step 5: Save the trained model to a file for later use
        model_filename = f'model_{symbol}.pkl'  # Save each model with the stock symbol name
        with open(model_filename, 'wb') as model_file:
            pickle.dump(model, model_file)  # Save the model to a .pkl file

    return predictions

# Step 5: Run the prediction process
df = await fetch_data()  # Fetch the data
predictions = prepare_lstm_data(df)  # Prepare and predict

# Print predictions for all stocks
print(predictions)

In [ ]:
from stockdata.models import HistoricalData
import pandas as pd
from asgiref.sync import sync_to_async
from sklearn.preprocessing import MinMaxScaler
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense
import numpy as np

# Step 1: Fetch data from the database asynchronously
async def fetch_data():
    # Fetch the stock data
    data = await sync_to_async(list)(HistoricalData.objects.values(
        'stock__symbol', 'date', 'open_price', 'close_price', 'high_price', 'low_price', 'volume'
    ))

    # Convert to pandas DataFrame
    df = pd.DataFrame(data)
    return df

# Step 2: Prepare data for each stock symbol
def prepare_lstm_data(df, time_step=60):
    stock_symbols = df['stock__symbol'].unique()
    predictions = {}

    for symbol in stock_symbols:
        stock_data = df[df['stock__symbol'] == symbol]
        close_prices = stock_data['close_price'].values.reshape(-1, 1)

        # Normalize data
        scaler = MinMaxScaler(feature_range=(0, 1))
        scaled_data = scaler.fit_transform(close_prices)

        # Prepare training data
        x_train, y_train = [], []
        for i in range(len(scaled_data) - time_step):
            x_train.append(scaled_data[i:i+time_step, 0])
            y_train.append(scaled_data[i + time_step, 0])
        
        x_train, y_train = np.array(x_train), np.array(y_train)
        x_train = x_train.reshape(x_train.shape[0], x_train.shape[1], 1)

        # Step 3: Build and train LSTM model for each stock
        model = Sequential()
        model.add(LSTM(50, return_sequences=True, input_shape=(x_train.shape[1], 1)))
        model.add(LSTM(50, return_sequences=False))
        model.add(Dense(25))
        model.add(Dense(1))

        model.compile(optimizer='adam', loss='mean_squared_error')
        model.fit(x_train, y_train, batch_size=1, epochs=1)

        # Step 4: Predict the next day's price for each stock
        last_60_days = scaled_data[-time_step:]
        last_60_days = last_60_days.reshape(1, -1, 1)
        predicted_price = model.predict(last_60_days)
        predicted_price_actual = scaler.inverse_transform(predicted_price)

        # Store predictions
        predictions[symbol] = predicted_price_actual[0][0]

        # Step 5: Save the trained model to a file for later use
        model_filename = f'model_{symbol}.pkl'  # Save each model with the stock symbol name
        with open(model_filename, 'wb') as model_file:
            pickle.dump(model, model_file)  # Save the model to a .pkl file

    return predictions

# Step 5: Run the prediction process
df = await fetch_data()  # Fetch the data
predictions = prepare_lstm_data(df)  # Prepare and predict

# Print predictions for all stocks
print(predictions)